<a href="https://colab.research.google.com/github/zxyup/ML_demo/blob/main/pytorch_cifar10_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [2]:
!ls
!du -h

sample_data
8.0K	./.config/configurations
72K	./.config/logs/2023.05.18
76K	./.config/logs
108K	./.config
55M	./sample_data
55M	.


In [5]:
import torchvision
train_data = torchvision.datasets.CIFAR10(root='/content/data', train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root='/content/data', train=False, transform=torchvision.transforms.ToTensor(),
                                         download=False)


100%|██████████| 170498071/170498071 [00:13<00:00, 13032183.06it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data


In [6]:
from torch.utils.data import DataLoader
# DataLoader加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
print("训练集的长度:{}".format(len(train_data)))
print("测试集的长度:{}".format(len(test_data)))


训练集的长度:50000
测试集的长度:10000


In [13]:
from torch import nn
# 搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model(x)
        return x


In [14]:
from torch import nn
model=Model().cuda()
# 损失函数
loss = nn.CrossEntropyLoss().cuda()

# 优化器
optimizer  = torch.optim.SGD(model.parameters(),lr=0.01,)

In [21]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# 准备数据集
train_data = torchvision.datasets.CIFAR10(root='/content/data', train=True, transform=torchvision.transforms.ToTensor(),
                                          download=False)
test_data = torchvision.datasets.CIFAR10(root='/content/data', train=False, transform=torchvision.transforms.ToTensor(),
                                         download=False)


print("训练集的长度:{}".format(len(train_data)))
print("测试集的长度:{}".format(len(test_data)))

# DataLoader加载数据集
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 搭建神经网络
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model(x)
        return x

# 创建网络模型
model = Model().cuda()

#添加tensorboard可视化数据
writer = SummaryWriter('../logs_tensorboard')

# 损失函数
loss = nn.CrossEntropyLoss().cuda()

# 优化器
optimizer  = torch.optim.SGD(model.parameters(),lr=0.01,)

i = 1 # 用于绘制测试集的tensorboard

# 开始循环训练
for epoch in range(30):
    num_time = 0 # 记录看看每轮有多少次训练
    print('开始第{}轮训练'.format(epoch+1))
    model.train() # 也可以不写，规范的话是写，用来表明训练步骤
    for data in train_dataloader:
        # 数据分开 一个是图片数据，一个是真实值
        imgs,targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        # 拿到预测值
        output = model(imgs)
        # 计算损失值
        loss_in = loss(output,targets)
        # 优化开始~ ~ 先梯度清零
        optimizer.zero_grad()
        # 反向传播+更新
        loss_in.backward()
        optimizer.step()
        num_time +=1

        if num_time % 100 == 0:
            writer.add_scalar('看一下训练集损失值',loss_in.item(),num_time)
            print('看一下训练集损失值',loss_in.item(),num_time)

    sum_loss = 0 # 记录总体损失值

    # 每轮训练完成跑一下测试数据看看情况
    accurate = 0
    model.eval() # 也可以不写，规范的话就写，用来表明是测试步骤
    with torch.no_grad():
        for data in test_dataloader:
            # 这里的每一次循环 都是一个minibatch  一次for循环里面有64个数据。
            imgs , targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            output = model(imgs)
            loss_in = loss(output,targets)

            sum_loss += loss_in
            # print('这里是output',output)
            accurate += (output.argmax(1) == targets).sum()

    print('第{}轮测试集的正确率:{:.2f}%'.format(epoch+1,accurate/len(test_data)*100))

    print('看一下测试集损失',sum_loss,i)
    print('看一下当前测试集正确率',accurate/len(test_data)*100,i)
    i +=1

    torch.save(model,'/content/model_pytorch/model_{}.pth'.format(epoch+1))
    print("第{}轮模型训练数据已保存".format(epoch+1))

writer.close()


训练集的长度:50000
测试集的长度:10000
开始第1轮训练
看一下训练集损失值 2.2812814712524414 100
看一下训练集损失值 2.2814927101135254 200
看一下训练集损失值 2.2672579288482666 300
看一下训练集损失值 2.2019999027252197 400
看一下训练集损失值 2.0847113132476807 500
看一下训练集损失值 2.015449285507202 600
看一下训练集损失值 1.9676933288574219 700
第1轮测试集的正确率:27.87%
看一下测试集损失 tensor(313.7956, device='cuda:0') 1
看一下当前测试集正确率 tensor(27.8700, device='cuda:0') 1
第1轮模型训练数据已保存
开始第2轮训练
看一下训练集损失值 1.8861925601959229 100
看一下训练集损失值 1.9201445579528809 200
看一下训练集损失值 1.8564902544021606 300
看一下训练集损失值 1.7857115268707275 400
看一下训练集损失值 1.816091537475586 500
看一下训练集损失值 1.8138777017593384 600
看一下训练集损失值 1.7865259647369385 700
第2轮测试集的正确率:33.11%
看一下测试集损失 tensor(291.6505, device='cuda:0') 2
看一下当前测试集正确率 tensor(33.1100, device='cuda:0') 2
第2轮模型训练数据已保存
开始第3轮训练
看一下训练集损失值 1.6409367322921753 100
看一下训练集损失值 1.6852794885635376 200
看一下训练集损失值 1.666130542755127 300
看一下训练集损失值 1.5800848007202148 400
看一下训练集损失值 1.644429326057434 500
看一下训练集损失值 1.678123950958252 600
看一下训练集损失值 1.7184823751449585 700
第3轮测试集的正确率:39.99